In [137]:
import numpy as np
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from sklearn.manifold.t_sne import _joint_probabilities, _kl_divergence
from sklearn.manifold._utils import _binary_search_perplexity
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform

In [138]:
def _joint_probabilities(distances, desired_perplexity):
    distances = distances.astype(np.float32, copy=False)
    conditional_P = _binary_search_perplexity(
        distances, None, desired_perplexity, False)
    P = conditional_P + conditional_P.T
    sum_P = np.maximum(np.sum(P), MACHINE_EPSILON)
    P = np.maximum(squareform(P) / sum_P, MACHINE_EPSILON)
    return P

In [139]:
MACHINE_EPSILON = np.finfo(np.double).eps

In [140]:
n_components = 2
perplexity = 30
learning_rate = 200
n_iter = 1000
min_grad_norm = 1e-7
n_iter_without_progress = 300
init = 'random'
random_state = np.random.mtrand._rand

In [141]:
def fit(X):
    n_samples = X.shape[0]
    
    neighbors_nn = None
    
    distances = pairwise_distances(X, metric='euclidean', squared=True)
    
    P = _joint_probabilities(distances, perplexity)
    
    X_embedded = 1e-4 * random_state.randn(n_samples, n_components).astype(np.float32)
    
    degrees_of_freedom = max(n_components - 1, 1)
    
    return _tsne(P, degrees_of_freedom, n_samples, X_embedded=X_embedded, neighbors=neighbors_nn)

In [142]:
def _kl_divergence(params, P, degrees_of_freedom, n_samples, n_components):
    X_embedded = params.reshape(n_samples, n_components)
    dist = pdist(X_embedded, "sqeuclidean")
    dist /= degrees_of_freedom
    dist += 1.
    dist **= (degrees_of_freedom + 1.0) / -2.0
    Q = np.maximum(dist / (2.0 * np.sum(dist)), MACHINE_EPSILON)
    
    kl_divergence = np.nan
    
    grad = np.ndarray((n_samples, n_components), dtype=params.dtype)
    PQd = squareform((P - Q) * dist)
    for i in range(n_samples):
        grad[i] = np.dot(np.ravel(PQd[i], order='K'),
                         X_embedded[i] - X_embedded)
    grad = grad.ravel()
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
    grad *= c

    return kl_divergence, grad

In [143]:
def _tsne(P, degrees_of_freedom, n_samples, X_embedded, neighbors=None):
    
#     args = {
#         'n_samples': n_samples,
#         'n_components': n_components
#     }

    opt_args = {
        'args': [P, degrees_of_freedom, n_samples, n_components]
    }

    params = X_embedded.ravel()
    
    obj_func = _kl_divergence
    
#     params, kl_divergence, it = _gradient_descent(obj_func, params, n_samples, n_components, degrees_of_freedom)
    params, kl_divergence, it = _gradient_descent(obj_func, params, **opt_args)
        
    X_embedded = params.reshape(n_samples, n_components)

    return X_embedded

In [144]:
def _gradient_descent(objective, p0, it=0, n_iter=1000,
                      n_iter_check=1, n_iter_without_progress=300,
                      momentum=0.8, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7, args=None, kwargs=None):
    
    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}
    
    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = i = it
    
    for i in range(it, n_iter):
        check_convergence = (i + 1) % n_iter_check == 0
        # only compute the error when needed
#         kwargs['compute_error'] = check_convergence or i == n_iter - 1

        error, grad = objective(p, *args, **kwargs)
#         error, grad = objective(p, degrees_of_freedom, n_samples, n_components)

        grad_norm = linalg.norm(grad)

        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, min_gain, np.inf, out=gains)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update
        
      print("[t-SNE] Iteration %d: error = %.7f,"
                  " gradient norm = %.7f"
                  % (i + 1, error, grad_norm))

        
        if error < best_error:
                best_error = error
                best_iter = i
        elif i - best_iter > n_iter_without_progress:
            break
        
        if grad_norm <= min_grad_norm:
            break

    return p, error, i

In [145]:
X, y = load_digits(return_X_y=True)

In [ ]:
fit(X)